In [1]:
import pdfplumber
import json
import psycopg2
import requests


In [2]:
# 📌 EXTRACTION DU TEXTE PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    return text

In [3]:
texte = extract_text_from_pdf("C:/Users/danie/OneDrive/Documents/GitHub/Challenge_web_mining/data/lm/lm_dan.pdf")
print(texte)

Daniella Rakotondratsimba
6 Rue Emile Duport 69009, Lyon
+33 7 86 62 53 12
daniellark11@gmail.com
Métropole de Lyon
Lyon, le 12 Février 2025
Objet : Candidature au poste de Stagiaire Chargé de mission Data – Data BI et Dataviz
Madame, Monsieur,
Actuellement en Master 2 Statistiques et Informatique pour la Science des Données (SISE) à
l’Université Lumière Lyon 2, je suis passionnée par l’analyse et la valorisation des données. Attirée par
les projets à impact direct sur la vie des citoyens, je souhaite rejoindre la Métropole de Lyon en tant
que stagiaire pour participer au développement de tableaux de bord analytiques. Disponible à partir
du 17 Mars 2025 pour une durée de 4 à 6 mois, je suis motivée à contribuer à vos initiatives
d’innovation numérique tout en renforçant mes compétences dans un environnement dynamique et
collaboratif.
Durant ma formation, j’ai acquis une solide expertise en business intelligence et gestion des
données. J’ai travaillé sur des projets impliquant la concep

In [ ]:
# 📌 APPEL API MISTRAL AVEC JSON STRICT
MISTRAL_API_KEY = "clé api"
API_URL = "https://api.mistral.ai/v1/chat/completions"

def query_mistral(prompt):
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistral-medium",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post(API_URL, headers=headers, json=data)

    if response.status_code == 200:
        try:
            return json.loads(response.json()["choices"][0]["message"]["content"])
        except json.JSONDecodeError:
            print("⚠️ Erreur : réponse Mistral non valide en JSON.")
            return None
    else:
        print("❌ Erreur API :", response.text)
        return None

In [5]:
# 📌 EXTRACTION DES DONNÉES AVEC FORMAT JSON STRICT
def extract_data(text):
    prompt = f"""
    Voici une lettre de motivation :
    "{text}"

    Formate ta réponse en JSON avec ces champs :
    {{
      "competences": ["compétence1", "compétence2"],
      "motivations": "motivation",
      "lieu": "ville"
    }}
    """
    return query_mistral(prompt)

In [8]:
data = extract_data(texte)
print(data)

{'competences': ['business intelligence', 'gestion de données', 'conception de tableaux de bord interactifs', 'préparation et traitement de données complexes', 'utilisation de Power BI, SQL et processus ETL', 'visualisation de données', 'compréhension des besoins utilisateurs', 'travail en équipe', 'adaptabilité'], 'motivations': "passionnée par l'analyse et la valorisation des données, intérêt pour les projets à impact direct sur la vie des citoyens, motivation à contribuer aux initiatives d'innovation numérique de la Métropole de Lyon, souhait de renforcer ses compétences dans un environnement dynamique et collaboratif", 'lieu': 'Lyon'}


In [10]:

# 📌 INSÉRER DANS POSTGRESQL
def insert_into_db(data):
    if not data:
        print("⚠️ Pas de données à insérer.")
        return

    try:
        conn = psycopg2.connect(
            dbname="test_lm",
            user="postgres",
            password="daniella",
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS lm (
                id SERIAL PRIMARY KEY,
                competences JSONB,
                motivations TEXT,
                lieu TEXT
            )
        """)

        cursor.execute("""
            INSERT INTO lm (competences, motivations, lieu)
            VALUES (%s, %s, %s)
        """, (json.dumps(data["competences"]), data["motivations"], data["lieu"]))

        conn.commit()
        print("✅ Données insérées avec succès !")

    except Exception as e:
        print("❌ Erreur PostgreSQL :", e)

    finally:
        cursor.close()
        conn.close()



In [ ]:
# 📌 EXÉCUTION DU PIPELINE
if data:
    print("✅ Compétences :", data["competences"])
    print("✅ Motivations :", data["motivations"])
    print("✅ Lieu :", data["lieu"])
    insert_into_db(data)

✅ Compétences : ['business intelligence', 'gestion de données', 'conception de tableaux de bord interactifs', 'préparation et traitement de données complexes', 'utilisation de Power BI, SQL et processus ETL', 'visualisation de données', 'compréhension des besoins utilisateurs', 'travail en équipe', 'adaptabilité']
✅ Motivations : passionnée par l'analyse et la valorisation des données, intérêt pour les projets à impact direct sur la vie des citoyens, motivation à contribuer aux initiatives d'innovation numérique de la Métropole de Lyon, souhait de renforcer ses compétences dans un environnement dynamique et collaboratif
✅ Lieu : Lyon
✅ Données insérées avec succès !
